# Initializing your environment

## Setting up a virtual environment
(Optional but recommended)

```bash
virtualenv -p python3 venv
source venv/bin/activate
```
(Use `deactivate` to exit from `source` once you are done)

Alternatively you can also prefix all your `python` and `pip` commands with `./venv/bin/` (e.g: `./venv/bin/pip3 install -U pip`)



## Setting up Jupyter

In order to follow along on your computer:

```bash
pip3 install notebook
jupyter-notebook
```

## Installation of PyMISP

#### Make sure the submodules are up-to-date and cloned

```bash
git submodule update --init --recursive PyMISP/
```

#### Install PyMISP with the developer options

```bash
cd PyMISP
pip3 install -e .
```

# Interacting with your MISP instance

### Recovering your API key

- Go to `Global Actions` then `My profile`
- Alternatively, access the `/users/view/me` URL of your MISP instance

## Initializing the variables

We need to set a few variables:
- The URL of the MISP instance
- Your API key
- The certificate verification bool variable

In [ ]:
# The URL of the MISP instance to connect to
misp_url = 'https://training.misp-community.org'

# The authentication key (can be found in the MISP
# web interface under _misp_url_/users/view/me -> Authkey)
misp_key = 'Al0Yg00EmIhlddsTqx1fAV9nBliw9RXdSe13OvST'

# Should PyMISP verify the MISP certificate
misp_verifycert = True

Now we can use the API class

In [ ]:
from pymisp import PyMISP

misp = PyMISP(misp_url, misp_key, misp_verifycert)

## Creating an Event

### Directly

In [ ]:
event1 = misp.add_event(
    {
        'info': 'CryptoLocker ransomware infection via e-mail',
        'distribution': 3,
        'analysis': 2,
        'threat_level_id': 1
    }
)

print(f'Event id of the created event: {event1.id}')

### Using the MISPEvent constructor

In [ ]:
from pymisp import MISPAttribute, MISPEvent

event = MISPEvent()
event.info = 'Ransomware infection via e-mail'
event.distribution = 3 # All communities
event.threat_level_id = 3 # High
event.analysis = 1 # Ongoing



### Add Attributes

In [ ]:
# Add the IP address attribute
ip_attribute = MISPAttribute()
ip_attribute.from_dict(
    **{
        'type': 'ip-dst',
        'value': '81.177.170.166',
        'comment': 'IP address of a C2 server distributing the ransomware'
    }
)
added_ip = event.add_attribute(**ip_attribute)
print(added_ip)

# Add the btc address attribute
btc_attribute = event.add_attribute('btc', '1J7fmT3Wv62u4p6FuwbVJQXhQEjw3jzV63',
                    comment='BTC address to pay the ransom')
print(btc_attribute)

### Add the Event

In [ ]:
misp_event = misp.add_event(event, pythonify=True)
print(f'Event id of the created event: {misp_event.id}')

## Updating an existing Event

### Fetching the Event

In [ ]:
# Pick an Event id - like the one you just printed above
event_id = 268

event = misp.get_event(event_id, pythonify=True)
print(event)

### Add some Objects

We want to push a file directly in our Event

In [ ]:
from pathlib import Path
test_path = Path().resolve().parent / 'PyMISP' / 'tests'
print(test_path)

from pymisp.tools import FileObject

file_object = FileObject(
    filepath=test_path / 'viper-test-files' / 'test_files' / 'EICAR.com',
    standalone=False
)

added_object = event.add_object(file_object)
print(added_object)

### Add new Objects

In [ ]:
from pymisp import MISPObject

# Encode the email received by the victim
email = MISPObject('email')
email.add_attribute('from', 'andrew_ryan@rindustries.rp')
email.add_attribute('subject', 'Report for case 4829-2375')
email.add_attribute('email-body', 'Please see the attached Iolta report for 4829-2375.\r\n\r\nWe received a check request in the amount of $19,637.28 for the above referenced file. However, the attached report refects a $0 balance. At your earliest convenience, please advise how this request is to be funded.\r\n\r\nThanks.\r\n\r\nAndrew_Ryan *\r\nAccounts Payable\r\n\r\nRyan Industries\r\n42, Central Control Hephaestus - Rapture\r\nwww.rindustries.rp\r\n\r\n*Not licensed to practise law.\r\n\r\nThis communication contains information that is intended only for the recipient named and may be privileged, confidential, subject to the attorney-client privilege, and/or exempt from disclosure under applicable law. If you are not the intended recipient or agent responsible for delivering this communication to the intended recipient, you are hereby notified that you have received this communication in error, and that any review, disclosure, dissemination, distribution, use, or copying of this communication is STRICTLY PROHIBITED. If you have received this communication in error, please notify us immediately by telephone at 1-800-766-7751 or 1-972-643-6600 and destroy the material in its entirety, whether in electronic or hard copy format.')
email_object = event.add_object(email)

In [ ]:
# Encode the Identity of the sender
person = MISPObject('person')
person.comment = 'The sender of the email'
person.add_attribute('full-name', 'Andrew Ryan')
person.add_attribute('e-mail', 'andrew_ryan@rindustries.rp')
person.add_attribute('role', 'Suspect')
person_object = event.add_object(person)

### Add references between objects

In [ ]:
# From the email object, add a reference to the person object
email_object.add_reference(person_object.uuid, 'sent-by')

# From the file object, add a reference to the email object
added_object.add_reference(email_object.uuid, 'dropped-by')

# From the file_object, add a reference to the IP address Attribute
added_object.add_reference(ip_attribute.uuid, 'connects-to')


### Add some tags

In [ ]:
# Adding tags to the event
event.add_tag('tlp:green')
event.add_tag('ransomware:infection="phishing-e=mails"')
event.add_tag('malware_classification:malware-category="Ransomware"')

# Adding a tag to the IP address attribute
added_ip.add_tag('adversary:infrastructure-type="C2"')

# Adding a Galaxy Cluster to the event
event.add_tag('misp-galaxy:ransomware="CryptoLocker"')

### Update the Event

We previously pushed an event to MISP directly, but we also did local changes.  
We need then to update the Event on MISP.

In [ ]:
event.analysis = 2 # Completed
event.publish()

misp.update_event(event)

## API operations

Here are some other useful operations available with the `PyMISP` constructor

### Direct call (no validation)

Since most of the operations are also available with direct calls:

In [ ]:
misp.direct_call(f'attributes/add/{event_id}', {'type': 'ip-dst', 'value': '10.9.8.7'})

### Searches

In [ ]:
import json

# Searching the Event index
endpoint = 'events/index'

body = {
    'org': 'ORGNAME',
    'published': True,
    'publish_timestamp': '1d'
}

response = misp.direct_call(endpoint, body)
print(json.dumps(response, indent=2))

### RestSearch

In [ ]:
endpoint = '/attributes/restSearch/'

body = {
    'returnFormat': 'json',
    'type': ['ip-src', 'ip-dst'],
    'last': '1d'
}

response = misp.direct_call(endpoint, body)
print(response)

## Using the search method

In [ ]:
# Search for Events published by the organisation 'ORGNAME' in the last 2 days
response = misp.search(org='ORGNAME', published=True, publish_timestamp='2d', pythonify=True)
print(response)

In [ ]:
# Search for Events containing Attributes with a specific value
response = misp.search(value='81.177.170.166', pythonify=True)
for event in response:
    print(event)

In [ ]:
# Search for published Events from the last 2 days tagged with tlp:clear
events = misp.search(publish_timestamp='2d', tags=['tlp:clear'], pythonify=True)
print(events)

### Search for attributes

In [ ]:
# Search for IP addresses from the last 2 days
attributes = misp.search(controller='attributes', type_attribute=['ip-src', 'ip-dst'], to_ids=1, last='2d', pythonify=True)
for attribute in attributes:
    print(attribute)

In [ ]:
# Search for Attributes with TLP tag, but not tlp:amber nor tlp:red
attributes = misp.search(controller='attributes', tags=['tlp:%', '!tlp:amber', '!tlp:red'], published=True, publish_timestamp='2d', pythonify=True)
for attribute in attributes:
    print(attribute)

In [ ]:
# Paginate on the results above
attributes = misp.search(
    controller='attributes', tags=['tlp:%', '!tlp:amber', '!tlp:red'], published=True, limit=5, page=1, publish_timestamp='2d', pythonify=True
)
for attribute in attributes:
    print(attribute)

### Trying different output formats

In [ ]:
# Returning the attributes in CSV
csv_attributes = misp.search(controller='attributes', type_attribute=['ip-src', 'ip-dst'], to_ids=1, last='2d', return_format='csv')
print(csv_attributes)

In [ ]:
# Converting data into STIX
stix2_content = misp.search(eventid=event_id, return_format="stix2")
print(json.dumps(stix2_content, indent=2))

In [ ]:
# Converting the Attributes only
stix2_content = misp.search(
    controller='attributes', type_attribute=["md5", "sha1", "sha256"], eventid=event_id,
    return_format="stix2", **{"stix-version": "2.1"}
)
print(json.dumps(stix2_content, indent=2))